In [1]:
from utils.utils import *
from scripts.config import DefaultArgsNamespace
import torch
import torch.nn as nn
import torchvision.models as models
from datautils.ems import *
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import StepLR
import wandb

import warnings
warnings.filterwarnings("ignore", message="Accurate seek is not implemented for pyav backend")




device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

args = DefaultArgsNamespace()

train_loader, val_loader, test_loader = eee_get_dataloaders(args)

# # get input feature  size
# dataiter = next(iter(train_loader))
# batch =  dataiter


# # Access the parsed arguments
model, optimizer, criterion, device = init_model(args)# verbose_mode = args.verbose
model = model.to(device)
scheduler = StepLR(optimizer, step_size=args.learning_params["lr_drop"], gamma=0.1)  # adjust parameters as needed

dummy_input = torch.randn(1, 30, 1024) # batch, num_frames, channels, height, width

dummy_input = dummy_input.to(args.device)

dummy_output = model(dummy_input)   

print(dummy_output.shape)




train dataset size:  5686
val dataset size:  1892
test dataset size:  2477


/scratch/cjh9fw/conda/egoexoems/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/scratch/cjh9fw/conda/egoexoems/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


x_shape torch.Size([1, 30, 1024])
torch.Size([1, 16])


In [2]:
def preprocess(x, modality, backbone):
    # check the shape of the input tensor
    output = None
    if('video' in modality):
        output = None
        x = x['frames']
        # extract resnet50 features
        x = x.to(device)
        x = backbone(x)
        output = x
    

    elif ( 'flow' in modality and  'rgb' in modality):

        # I3D features are already extracted
        flow = x['flow'].float()
        rgb = x['rgb'].float()
        output = torch.cat((flow, rgb), dim=-1).float()

    elif ('rgb' in modality):
        # I3D features are already extracted
        output = x['rgb'].float()

    elif ('flow' in modality):
        # I3D features are already extracted
        output = x['flow'].float()

    elif ('audio' in modality):
        # Audio features are already extracted
        output = x['audio'].float()

    feature_size = output.shape[-1]
    return output, feature_size

In [4]:
print("Modality: ", args.dataloader_params['modality'])

# Find feature dimension
input, feature_size = preprocess(next(iter(train_loader)), args.dataloader_params['modality'], model.extract_resnet)

print("Feature size: ", feature_size)

args.transformer_params['input_dim'] = feature_size

model, optimizer, criterion, device = init_model(args)# verbose_mode = args.verbose
model = model.to(device)
scheduler = StepLR(optimizer, step_size=args.learning_params["lr_drop"], gamma=0.1)  # adjust parameters as needed


# # print one batch
for i, batch in enumerate(train_loader):
    input,feature_size = preprocess(batch, args.dataloader_params['modality'], model.extract_resnet)
    print(input.shape)
    output = model(input)
    print(output.shape)
    break

# # Train the model
# for epoch in range(args.learning_params["epochs"]):
#     train_loss = train_one_epoch(model, train_loader, criterion, optimizer, device, wandb_logger)
#     val_loss = validate(model, val_loader, criterion, device, wandb_logger)
#     scheduler.step()
#     print(f"Epoch: {epoch}, Train Loss: {train_loss}, Val Loss: {val_loss}")


Modality:  ['video', 'audio']
Item index: 101, Clip index: 6
Feature size:  2048


/scratch/cjh9fw/conda/egoexoems/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/scratch/cjh9fw/conda/egoexoems/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Item index: 587, Clip index: 2
torch.Size([1, 30, 2048])
x_shape torch.Size([1, 30, 2048])
torch.Size([1, 16])
